In [9]:
#HIDDEN
from ipywidgets import interact, interactive, fixed, interact_manual

import ipywidgets as widgets

from ipywidgets import FloatSlider

from ipywidgets import VBox, HBox, interactive_output

from ipywidgets import Dropdown
from matplotlib.image import imread



# Part I : Cloud Map Over Time

In [10]:
#HIDDEN
import pickle

In [11]:
#HIDDEN
#load data
data = pickle.load(open('data_project.p', 'rb'))

In [14]:
#HIDDEN
from matplotlib import pyplot as plt
from matplotlib.pylab import subplots
from wordcloud import WordCloud
import numpy as np
from random import shuffle
from ipywidgets import IntSlider
import pandas as pd
from matplotlib.image import imread

year_end=2019
year_start=2015
month=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept','Oct','Nov','Dec']
start_month=1
end_month=(year_end-year_start+1)*12
#fsx = IntSlider(min=start_month,max=end_month,step=1,continuous_update=False,description='Time:') #slider of months

time_idx=pd.date_range(start=str(year_start)+'0101',end=str(year_end)+'1231',freq='M')
fmt='%Y-%m'
options = [(item.strftime(fmt),item) for item in time_idx]

fsx=widgets.SelectionSlider(
            description='Date',
            options=options,
            continuous_update=False
        )

status = widgets.Output() #status indicator
status2 = widgets.Output() #status indicator for part 2
status3 = widgets.Output() #status indicator for part 3
#with status:
    #print('Please select a date'+'\r',end='')

def fplot(x,Genre):
    """x : input time
     Genre: different topics"""
    #fig,ax=subplots()
    #ax.plot(xi,np.sin(2*np.pi*x*xi+y),color=color)
    #indicate running
    plt.cla()
    loading="Now loading..."
    plt.axis('off') 
    plt.text(0.5,0.5,loading)
    #load_img=imread('loading_img.jpg')
    #plt.imshow(load_img)
    #plt.show()
    #print(loading+'\r',end='')
    with status:
        print('Now loading...'+'\r',end='')
    
    data_cloud=data[Genre]
    freq=np.array(list(data_cloud.frequency_time))
    string = ''
    m,n=freq.shape
    for i in range(m):
        time=str(x).split(' ')[0].split('-')
        col=(int(time[0])-year_start)*12+int(time[1])
        string=string+(data_cloud.loc[i].words+' ')*(freq[i,col-1])
    if string.split()==[]:
        """if there is no words shown"""
        string='There is no interesting word'
        plt.cla()
        plt.axis('off') 
        plt.text(0.5,0.5,string)
        #plt.imshow(nointerest_img)
        plt.show()       
        #print(string)
        with status:
            print('Complete!     '+'\r',end='')
        return
    lis=string.split(' ')
    shuffle(lis)
    string=' '.join(lis)
    wc = WordCloud(
               background_color='white',max_words=20,
               width=1000,
               height=1000,
               ).generate(string)  #wordcloud
    #wc.to_file('ss.png') 
    plt.cla()
    plt.imshow(wc)  
    plt.axis('off') 
    plt.show()
    #print(string)
    with status:
        print('Complete!     '+'\r',end='')
interactive_plot = interactive(fplot,x=fsx,Genre=['cooking', 'influencers', 'gaming'])
#display(interactive_plot)

In [15]:
#HIDDEN
def acrosstime(dis_num,Genre):   
    data_cloud=data[Genre]
    freq=np.array(list(data_cloud.frequency_time))
    freq=np.sum(freq,axis=1)
    string = ''
    idx=np.argpartition(freq,-dis_num)[-dis_num:]
    string=''
    for i in idx:
        string=string+(data_cloud.loc[i].words+' ')*(freq[i])
    if string.split()==[]:
        """if there is no words shown"""
        string='There is no interesting word'
        plt.cla()
        plt.axis('off') 
        plt.text(0.5,0.5,string)
        #plt.imshow(nointerest_img)
        plt.show()       
        #print(string)
        return
    lis=string.split(' ')
    shuffle(lis)
    string=' '.join(lis)
    wc = WordCloud(
               background_color='white',max_words=dis_num,
               width=1000,
               height=1000,
               ).generate(string)  #wordcloud
    #wc.to_file('ss.png') 
    plt.cla()
    plt.imshow(wc)  
    plt.axis('off') 
    plt.show()
    #print(string)
dis_num=widgets.BoundedIntText(
    value=10,
    min=0,
    max=200,
    step=1,
    description='Integer:',
    disabled=False
)
acrossAlltime=interactive(acrosstime,dis_num=dis_num,Genre=['cooking', 'influencers', 'gaming'])

# Part II: Like and Dislike

In [16]:
#HIDDEN
Genre= Dropdown(options=['cooking', 'influencers', 'gaming'])
#Fsx = IntSlider(min=start_month,max=end_month,step=1,continuous_update=False,description='Time:')
Fsx=widgets.SelectionSlider(
            description='Date',
            options=options,
            continuous_update=False
        )
loaded={}
loaded['like']=0
loaded['hate']=0
def Fplot(x,Genre,like):
    """x: input time
      Genre: diffenrent topics
      like: positive or negative result"""
    #fig,ax=subplots()
    #ax.plot(xi,np.sin(2*np.pi*x*xi+y),color=color)
    plt.cla()
    loading="Now loading..."
    plt.axis('off') 
    plt.text(0.5,0.5,loading)
    if loaded['like']==0 and loaded['hate']==0:
        with status2:
            print('Now loading...'+'\r',end='')
    
    data_cloud=data[Genre]
    if like=='like':
        data_cloud=data_cloud[data_cloud.polarity>0]#data_cloud[data_cloud.likes_mean >= data_cloud.dislikes_mean]
    elif like=='hate':
        data_cloud=data_cloud[data_cloud.polarity<=0]
    data_cloud=data_cloud.reset_index()
    freq=np.array(list(data_cloud.frequency_time))
    string = ''
    if freq.shape==(0,):
        string='There is no interesting word'
        plt.cla()
        plt.axis('off') 
        plt.text(0.5,0.5,string)
        #print(string)
        loaded[like]=1
        if loaded['like']==1 and loaded['hate']==1:
            with status2:
                print('Complete!     '+'\r',end='')
            loaded['like']=0
            loaded['hate']=0
        return
        
    m,n=freq.shape
    for i in range(m):
        time=str(x).split(' ')[0].split('-')
        col=(int(time[0])-year_start)*12+int(time[1])
        string=string+(data_cloud.loc[i].words+' ')*(freq[i,col-1])
    if string.split()==[]:
        """if there is no words shown"""
        string='There is no interesting word'
        plt.cla()
        plt.axis('off') 
        plt.text(0.5,0.5,string)
        #print(string)
        loaded[like]=1
        if loaded['like']==1 and loaded['hate']==1:
            with status2:
                print('Complete!     '+'\r',end='')
            loaded['like']=0
            loaded['hate']=0
        return
    lis=string.split(' ')
    shuffle(lis)
    string=' '.join(lis)
    wc = WordCloud(
               background_color='white',max_words=20,
               width=1000,
               height=1000,
               ).generate(string)  #wordcloud
    #wc.to_file('ss.png') 
    plt.cla()
    plt.imshow(wc)  
    plt.axis('off') 
    plt.show()
    #print(string)
    loaded[like]=1
    if loaded['like']==1 and loaded['hate']==1:
        with status2:
            print('Complete!     '+'\r',end='')
        loaded['like']=0
        loaded['hate']=0
#l = widgets.link((sliders1, 'value'), (slider2, 'value'))
interactive_plot_like = interactive(Fplot,x=Fsx,Genre=Genre,like='like')
interactive_plot_hate=interactive(Fplot,x=Fsx,Genre=Genre,like='hate')
#display(HBox([interactive_plot_like,interactive_plot_hate ]))

# Part III: Text Analysis 

In [17]:
#HIDDEN
def analyze_text(text, genre, metric):
    '''
    Purpose: 
    Analyze the given text and produce color labels for the words as well as generate an overall score based on the
    given genre and metric
    
    Input:
    text = str; scalar depicting the text that needs to be analyzed
    genre = str; scalar depicting the genre of the content: "cooking", "gaming", "influencers"
    metric = str; scalar depicting the metric to base the analysis on: "likes_mean", "likes_median",
             "dislikes_mean", "dislikes_median", "views_mean", "views_median", "polarity", "subjectivity"
    
    Output:
    analysis = list; a list with the same number of elements as number of words in given text, with each
               corresponding element being the color for that word: "red" means bad, "yellow" means okay, "green"
               means good and "white" means "Not found" (in database)
    score_avg = float; average value of the score: float or "Not applicable" (if none of the words matched the
                database)
               
    '''
    from textblob import TextBlob
    import pickle
    pickle.load(open('data_project.p', 'rb'))
    Blob = TextBlob(text)
    scores = []
    df_genre = data[genre]
    score_avg = 0
    counter = 0
    for word in list(Blob.words):
        if word in list(df_genre.words):
            word_index = df_genre.words[df_genre.words==word].index[0]
            scores.append(df_genre[metric][word_index])
            score_avg += df_genre[metric][word_index]
            counter += 1
        else:
            scores.append("Not found")
    if counter > 0:
        score_avg = score_avg/counter
    else:
        score_avg = "Not Applicable"
    intervals = [df_genre[metric].mean()-df_genre[metric].std(), df_genre[metric].mean()+df_genre[metric].std()]
    analysis = []
    for score in scores:
        if score=="Not found":
            analysis.append("white")
            continue
        if score<=intervals[0]:
            analysis.append("red")
        elif score>intervals[0] and score<=intervals[1]:
            analysis.append("yellow");
        elif score>intervals[1]:
            analysis.append("green")
    return analysis, score_avg
w=widgets.Textarea(
    value='',
    placeholder='Type to analyze',
    description='Text box:',
    disabled=False,
    continuous_update=False
)
def color_changer(x,Genre,Metric):
    with status3:
        print('Now loading...'+'\r',end='')
    categorization, overall_score = analyze_text(x,Genre,Metric)
    words = x.split()
    for word in words:
        color,score = analyze_text(word,Genre,Metric)
        if color[0] == "red":
            print('\033[31m %s' %word, end = "")
        elif color[0] == "yellow":
            print('\033[33m %s' %word, end = "")
        elif color[0] == "green":
            print('\033[32m %s' %word, end = "")
        else:
            print('\033[37m %s' %word, end = "")
    with status3:
        print('Complete!     '+'\r',end='')
box = interactive(color_changer,x=w,Genre=['cooking', 'influencers', 'gaming'],Metric=["likes_mean", "likes_median",
             "dislikes_mean", "dislikes_median", "views_mean", "views_median", "polarity", "subjectivity"])

# Overall

## Displays below will be presented in the website.

In [18]:
#HIDDEN
#description of all three parts
descrip_part1="""
In this part, you can use our tools to create a word cloud of popular words in different time periods. Size of a word will be based on
its popularity. 
 We also provide you with different genres to choose. You can check the popular words within the genre you have interest in.
 The time will be in a month and year format that the user can toggle with the slider and see how the word cloud changes over time.
"""
descrip_part2="""
A second display will compose of two word clouds that present the positive and negative words respectively, 
showcasing the words’ popularity based on its size. 
"""
descrip_part3="""
Our Text Analysis tool will generate word connotations based on the words typed into the textbox.  
Words will be colored according to its positive or negative sentiment.
 For example, words in red denoting a negative word, words in yellow denoting neutral words and green denoting a positive word. Words that can not be 
 decided will remain in black.
 Also, you can select the genre and metric of interest. 
"""
Descrip_part1=widgets.HTML(value = f"<b><font size='2' color='Black'>{descrip_part1}</b>")
Descrip_part2=widgets.HTML(value = f"<b><font size='2' color='Black'>{descrip_part2}</b>")
Descrip_part3=widgets.HTML(value = f"<b><font size='2' color='Black'>{descrip_part3}</b>")

In [19]:
#HIDDEN
tab = widgets.Tab()
tab.children = [VBox([Descrip_part1,interactive_plot,status]),acrossAlltime]
tab.set_title(0, 'Popular Keywords VS Month')
tab.set_title(1, 'Popular Keywords across all time')

In [20]:
#HIDDEN
#folder for different part
accordion = widgets.Accordion(children=[VBox([Descrip_part1,interactive_plot,status]),
                                        VBox([Descrip_part2,
                                              HBox([interactive_plot_like,interactive_plot_hate ]),status2]),
                                        VBox([Descrip_part3,box,status3])])
accordion.set_title(0, 'Popular Keywords over Time')
accordion.set_title(1, 'Likes&Dislikes over Time ')
accordion.set_title(2, 'Text Analysis ')

In [22]:
#HIDDEN
#container for the main interface
from ipywidgets import AppLayout, Layout
title = 'Youtube Trend over Time'
Title = widgets.HTML(value = f"<b><font size='5' color='Black'>{title}</b>")
descrip="""How insane is it that sometimes you see the most amazing video on youtube, 
and yet it took nearly a decade for it to be popular? Or a subpar video which hits a 
million views within a week? A lot of it has to do with the way you market your content. 
Sponsoring advertisements for your posts helps, but it depends heavily on the 
textual supporting descriptions that you add. And an even deeper question is: what makes certain 
words more popular than others? What receives more hate by the public? Are certain 
topics more active during election years or during a pandemic? Fortunately, our platform and tools will figure it out for you
. We’re trying to create a simple 
platform which serves as a proof of concept for how sentiment analysis can be extended 
to utility on a daily basis, with a very wide customer demographic.
"""
Descrip=widgets.HTML(value = f"<b><font size='2' color='Black'>{descrip}</b>")
AppLayout(header=VBox([Title,Descrip]),
          left_sidebar=None,
          center=accordion,
          right_sidebar=None,
          footer=None)
          
          #widgets.IntSlider(description='c',
                                   #layout=Layout(height='auto', width='auto')))

AppLayout(children=(VBox(children=(HTML(value="<b><font size='5' color='Black'>Youtube Trend over Time</b>"), …